<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-18 10:49:19
-------------------
qualified stocks: 93
with latest results: 28
still star stocks: 18
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  2.51 L
Current:  1.37 C
-------------------
Today PnL: 2.47 L (1.84%)
Current PnL: -18.30 L (-12.45%)
CY Booked + Current PnL: -10.03 L (-6.82%)
-------------------
Total profit:  3.58 L
Total loss:  -21.88 L
-------------------
Total Booked + Current PnL: 16.95 L (14.12%)
Total Booked PnL: 35.25 L (29.36%)
Curr Year Booked PnL: 8.27 L (6.06%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.14 C
Est FTT PnL: 77.77 L (56.94%)
Deployed:  1.20 C
Current:  1.37 C
CAGR/XIRR %: 8.27%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,HINDALCO,651.95,756.01,-17.42,H-LC,6.55,114032.0,9720.0,6933.0,2.51,9.32,6.08,15.96,11.0,1.40,0.85,27.79,X5K,ATH,METALS
51,LALPATHLAB,2838.53,3545.00,-22.21,H-SC,10.92,232342.0,39322.0,8713.0,2.06,20.37,3.75,24.89,134.0,4.51,1.73,48.47,X40N,NTT,HEALTHCARE
4,APOLLOHOSP,7098.95,8285.00,-13.20,H-LC,6.28,173536.0,17359.0,8729.0,0.85,11.12,5.03,16.71,19.0,1.99,1.29,30.32,X40N,BTT,HEALTHCARE
68,SBIN,760.30,863.00,-11.34,M-LC,3.86,219632.0,18152.0,9071.0,0.27,9.01,4.13,13.51,61.0,2.00,1.64,22.74,XY25,NTT,BANKS
55,MEDANTA,1087.93,1486.00,10.37,H-MC,13.91,157609.0,31409.0,14768.0,-1.12,24.89,9.37,36.59,111.0,2.13,1.18,40.52,XY24,NTT,HEALTHCARE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,RAJOOENG,102.50,143.33,-31.84,H-SC,6.00,103220.0,720.0,40111.0,0.64,0.70,38.86,39.83,116.0,0.02,0.77,2.20,AR,ATH,MISC
58,PGHH,13388.00,18062.58,-30.28,H-MC,6.24,201780.0,960.0,69150.0,1.39,0.48,34.27,34.92,82.0,0.01,1.51,5.18,X40,ATH,FMCG
56,NATIONALUM,189.63,247.44,-49.46,H-MC,1.99,100218.0,-286.0,30927.0,0.81,-0.28,30.86,30.49,80.0,-0.01,0.75,32.03,MH,ATH,METALS


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,CIPLA,1492.70,1795.0,-19.78,H-LC,2.80,214049.0,9549.0,31872.0,-0.10,4.67,14.89,20.25,10.0,0.30,1.60,13.81,X40N,BTT,PHARMA
57,NESTLEIND,1134.30,1377.0,-11.66,H-LC,3.36,271674.0,6248.0,50531.0,6.57,2.35,18.60,21.40,8.0,0.12,2.03,9.61,XY25,NTT,FMCG
5,ASIANPAINT,2961.56,4250.0,-10.05,H-LC,3.71,175338.0,-26048.0,113654.0,1.97,-12.93,64.82,43.51,27.0,-0.23,1.31,22.05,X40,BTT,PAINTS
53,LTIM,5564.16,7230.2,-7.23,H-LC,3.73,183978.0,-16332.0,76314.0,0.05,-8.15,41.48,29.94,17.0,-0.21,1.37,27.10,X200,ATH,IT
37,HINDUNILVR,2413.81,2723.0,-8.54,H-LC,5.93,287571.0,17224.0,17398.0,3.51,6.37,6.05,12.81,28.0,0.99,2.14,20.27,XY25,NTT,FMCG


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,RELAXO,902.64,1176.00,-33.88,H-SC,10.63,61350.0,-54188.0,89178.0,9.78,-46.90,145.36,30.28,136.0,-0.61,0.46,19.85,X40N,NTT,FOOTWEAR
90,VOLTAS,1278.28,1918.49,-1.64,H-MC,7.59,206415.0,14673.0,81369.0,7.95,7.65,39.42,50.08,101.0,0.18,1.54,14.97,XY25,ATH,AC
15,BLUESTARCO,1646.70,2326.38,11.16,H-MC,6.57,191840.0,27170.0,40804.0,7.46,16.50,21.27,41.28,91.0,0.67,1.43,25.80,X40N,ATH,AC
57,NESTLEIND,1134.30,1377.00,-11.66,H-LC,3.36,271674.0,6248.0,50531.0,6.57,2.35,18.60,21.40,8.0,0.12,2.03,9.61,XY25,NTT,FMCG
13,BATAINDIA,1550.24,2096.00,-38.01,M-SC,7.49,92670.0,-36000.0,81299.0,6.36,-27.98,87.73,35.20,209.0,-0.44,0.69,6.36,X40,NTT,FOOTWEAR


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
29,EASEMYTRIP,18.12,26.40,-8.34,M-SC,25.38,88212.0,-88603.0,169402.0,-1.63,-50.11,192.04,45.70,197.0,-0.52,0.66,2.61,XY24,NTT,TRAVEL
35,HEROMOTOCO,4311.81,5949.07,-11.08,H-MC,7.83,164784.0,13871.0,43437.0,-1.27,9.19,26.36,37.97,92.0,0.32,1.23,36.82,AR,ATH,AUTO
55,MEDANTA,1087.93,1486.00,10.37,H-MC,13.91,157609.0,31409.0,14768.0,-1.12,24.89,9.37,36.59,111.0,2.13,1.18,40.52,XY24,NTT,HEALTHCARE
73,SONACOMS,578.05,1006.00,-37.09,M-SC,13.60,78304.0,-22855.0,97747.0,-0.97,-22.59,124.83,74.03,203.0,-0.23,0.58,10.42,AR,BTT,AUTO
34,HCLTECH,1562.67,1943.91,1.94,H-LC,10.47,180706.0,-9940.0,56453.0,-0.58,-5.21,31.24,24.40,9.0,-0.18,1.35,9.89,X40,ATH,IT


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TTKPRESTIG,769.29,770.0,95.30,M-SC,10.14,84141.0,-16636.0,16727.0,1.68,-16.51,19.88,0.09,243.0,-0.99,0.63,10.54,OX40N,NTT,DURABLES
20,CERA,7989.07,9475.0,-17.90,H-SC,3.14,117261.0,-26542.0,53283.0,1.82,-18.46,45.44,18.60,148.0,-0.50,0.87,29.40,OX40N,NTT,CERAMICS
43,INDIGOPNTS,1407.73,1408.0,114.84,M-SC,2.56,140839.0,-33720.0,33759.0,2.58,-19.32,23.97,0.02,221.0,-1.00,1.05,22.13,OX40N,NTT,PAINTS
84,UJJIVANSFB,52.77,60.0,80.86,H-SC,20.11,114048.0,-28431.0,47957.0,1.37,-19.95,42.05,13.70,163.0,-0.59,0.85,34.61,OX40N,NTT,BANKS
49,KANSAINER,299.63,340.0,-68.89,H-SC,3.02,214371.0,-55296.0,91622.0,1.16,-20.51,42.74,13.47,142.0,-0.60,1.60,8.43,XY24,NTT,PAINTS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TCS,3794.03,4998.00,-28.69,H-LC,16.10,235537.0,-60397.0,154300.0,-0.09,-20.41,65.51,31.73,2.0,-0.39,1.76,0.56,X40,BTT,IT
45,INFY,1461.46,2275.00,-21.39,H-LC,11.25,257850.0,-3751.0,149373.0,-0.50,-1.43,57.93,55.67,4.0,-0.03,1.92,4.58,X40,BTT,IT
89,VBL,492.64,671.64,-11.46,H-LC,10.05,268677.0,3637.0,92667.0,-0.49,1.37,34.49,36.33,5.0,0.04,2.00,14.85,X40N,ATH,FMCG
1,ABB,5551.76,7934.00,-42.01,H-LC,10.80,197086.0,-19433.0,112339.0,0.47,-8.98,57.00,42.91,7.0,-0.17,1.47,3.21,AR,NTT,ELECTRICAL
57,NESTLEIND,1134.30,1377.00,-11.66,H-LC,3.36,271674.0,6248.0,50531.0,6.57,2.35,18.60,21.40,8.0,0.12,2.03,9.61,XY25,NTT,FMCG


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LALPATHLAB,2838.53,3545.0,-22.21,H-SC,10.92,232342.0,39322.0,8713.0,2.06,20.37,3.75,24.89,134.0,4.51,1.73,48.47,X40N,NTT,HEALTHCARE
55,MEDANTA,1087.93,1486.0,10.37,H-MC,13.91,157609.0,31409.0,14768.0,-1.12,24.89,9.37,36.59,111.0,2.13,1.18,40.52,XY24,NTT,HEALTHCARE
28,DMART,3764.92,5201.0,-10.86,H-LC,6.66,222627.0,38146.0,32214.0,4.25,20.68,14.47,38.14,35.0,1.18,1.66,33.47,X40N,NTT,FMCG


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,BLUESTARCO,1646.70,2326.38,11.16,H-MC,6.57,191840.0,27170.0,40804.0,7.46,16.50,21.27,41.28,91.0,0.67,1.43,25.80,X40N,ATH,AC
50,KPIGREEN,497.21,731.35,10.56,H-SC,11.71,131922.0,6625.0,52373.0,2.92,5.29,39.70,47.09,141.0,0.13,0.98,65.11,MH,ATH,POWER
55,MEDANTA,1087.93,1486.00,10.37,H-MC,13.91,157609.0,31409.0,14768.0,-1.12,24.89,9.37,36.59,111.0,2.13,1.18,40.52,XY24,NTT,HEALTHCARE
90,VOLTAS,1278.28,1918.49,-1.64,H-MC,7.59,206415.0,14673.0,81369.0,7.95,7.65,39.42,50.08,101.0,0.18,1.54,14.97,XY25,ATH,AC
19,CAMS,3643.00,5211.76,-4.05,H-SC,4.19,109203.0,7199.0,36725.0,0.81,7.06,33.63,43.06,123.0,0.20,0.81,25.38,X40N,ATH,MISC


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-11.08,H-MC,7.83,164784.0,13871.0,43437.0,-1.27,9.19,26.36,37.97,92.0,0.32,1.23,36.82,AR,ATH,AUTO
50,KPIGREEN,497.21,731.35,10.56,H-SC,11.71,131922.0,6625.0,52373.0,2.92,5.29,39.70,47.09,141.0,0.13,0.98,65.11,MH,ATH,POWER
42,INDIAMART,2327.09,4850.92,-51.84,H-SC,11.79,134493.0,11157.0,122604.0,-0.32,9.05,91.16,108.45,119.0,0.09,1.00,33.42,AR,ATH,MISC
91,WHIRLPOOL,1219.98,2270.00,-42.43,M-SC,4.53,97320.0,5822.0,72932.0,4.59,6.36,74.94,86.07,219.0,0.08,0.73,40.78,XR,NTT,DURABLES
61,RAJOOENG,102.50,143.33,-31.84,H-SC,6.00,103220.0,720.0,40111.0,0.64,0.70,38.86,39.83,116.0,0.02,0.77,2.20,AR,ATH,MISC


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-49.70,L-SC,21.40,73709.0,-39840.0,79893.0,1.13,-35.09,108.39,35.27,268.0,-0.50,0.55,80.40,XR,NTT,HOTELS
3,ALKYLAMINE,2347.98,4546.37,-1.48,H-SC,15.43,91698.0,-9265.0,103793.0,2.86,-9.18,113.19,93.63,149.0,-0.09,0.68,36.58,SR,ATH,CHEMICALS
54,MASFIN,326.60,399.50,-17.80,H-SC,13.29,93510.0,-4470.0,26342.0,3.73,-4.56,28.17,22.32,152.0,-0.17,0.70,35.62,XR,ATH,FINANCE
88,VALIANTORG,512.64,838.00,-326.45,H-SC,16.43,96200.0,-37086.0,121683.0,0.39,-27.82,126.49,63.47,143.0,-0.30,0.72,61.26,XR,NTT,CHEMICALS
9,BAJAJFINSV,1994.80,2758.00,-8.22,H-LC,7.23,98412.0,667.0,36727.0,4.33,0.68,37.32,38.26,38.0,0.02,0.73,29.55,X40,BTT,FINANCE


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-11.08,H-MC,7.83,164784.0,13871.0,43437.0,-1.27,9.19,26.36,37.97,92.0,0.32,1.23,36.82,AR,ATH,AUTO
51,LALPATHLAB,2838.53,3545.00,-22.21,H-SC,10.92,232342.0,39322.0,8713.0,2.06,20.37,3.75,24.89,134.0,4.51,1.73,48.47,X40N,NTT,HEALTHCARE
28,DMART,3764.92,5201.00,-10.86,H-LC,6.66,222627.0,38146.0,32214.0,4.25,20.68,14.47,38.14,35.0,1.18,1.66,33.47,X40N,NTT,FMCG
5,ASIANPAINT,2961.56,4250.00,-10.05,H-LC,3.71,175338.0,-26048.0,113654.0,1.97,-12.93,64.82,43.51,27.0,-0.23,1.31,22.05,X40,BTT,PAINTS
4,APOLLOHOSP,7098.95,8285.00,-13.20,H-LC,6.28,173536.0,17359.0,8729.0,0.85,11.12,5.03,16.71,19.0,1.99,1.29,30.32,X40N,BTT,HEALTHCARE


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.56
1,25,41.39
2,50,71.21


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    38.93
LC    34.75
MC    26.31
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.47
X40N     15.54
X40      13.26
XY25     11.71
XR        8.95
AR        8.23
OX40N     7.38
X5K       2.24
MH        1.73
X200      1.37
SR        1.11
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    26.56
H-SC    24.25
H-MC    23.38
M-SC    13.22
M-LC     7.13
M-MC     2.59
L-SC     1.46
L-LC     1.06
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.17,-1.31,33.87
IT,11.72,-18.65,77.90
FINANCE,8.17,-18.87,62.85
BANKS,7.51,-13.40,61.55
MISC,5.78,-18.51,86.43
PAINTS,5.60,-16.39,39.33
HEALTHCARE,5.25,6.84,16.68
ELECTRICAL,4.70,-11.52,59.81
AUTO,4.40,-14.53,64.37


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2823549.0,23
AR,1105174.0,10
XR,1094546.0,13
X40,795126.0,10
X40N,521928.0,12
XY25,478008.0,8
OX40N,467584.0,10
SR,193748.0,2
X5K,137842.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2762492.0,28
M-SC,1474408.0,17
H-MC,1245127.0,18
H-LC,1174276.0,18
M-LC,422316.0,5
M-MC,331834.0,2
L-SC,265160.0,3
L-MC,57512.0,1
L-LC,43994.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      928972.0      6
M-SC       XY24      862199.0      7
H-SC       AR        668527.0      5
           XR        517268.0      6
H-LC       X40       510507.0      5
H-MC       XY24      485562.0      5
M-MC       XY24      331834.0      2
H-LC       AR        232680.0      2
           X40N      208425.0      5
H-SC       OX40N     204583.0      4
H-MC       X40       203320.0      4
           XY25      202363.0      2
H-SC       X40N      197021.0      4
           SR        193748.0      2
L-SC       XR        175246.0      2
M-LC       XY24      175088.0      2
M-SC       AR        160530.0      2
M-LC       X5K       130909.0      1
M-SC       OX40N     125414.0      4
           XY25      123057.0      1
           XR        121909.0      2
H-MC       X40N      116482.0      3
           XR        115363.0      1
M-LC       XR        107248.0      1
H-LC       XY25       99523.0      3
L-SC       OX40N      89914.0      1
M-SC       X40        81299.0      1
H-LC       X200       76314.0      1
L-MC       XR         57512.0      1
H-SC       MH         52373.0      1
H-MC       OX40N      47673.0      1
L-LC       XY25       43994.0      1
H-MC       AR         43437.0      1
H-LC       XY24       39894.0      1
H-MC       MH         30927.0      1
M-LC       XY25        9071.0      1
H-LC       X5K         6933.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
